## Deep Learning for Fertility Prediction (DLMP)

### Import packages 

In [1]:
import tensorflow as tf
import numpy as np
import os as os
tfkl = tf.keras.layers
import random

### Import functions

In [20]:
import training_functions
import importlib

importlib.reload(training_functions)

<module 'training_functions' from '/Users/paigepark/Library/Mobile Documents/com~apple~CloudDocs/repos/deep-fert/code/training_functions.py'>

### Import data

In [3]:
asfr_training = np.loadtxt('../data/asfr_training.txt')
asfr_test = np.loadtxt('../data/asfr_test.txt')

In [4]:
asfr_training.shape

(75936, 4)

In [5]:
geos_key = np.load('../data/geos_key.npy')

#### All Country Model

These models are those used in the paper to produce all of main figures/table in the paper.

In [9]:
# prep data
train_prepped = training_functions.prep_data(asfr_training, mode="train", changeratetolog=False)
test_prepped = training_functions.prep_data(asfr_test, mode="test", changeratetolog=False)

In [10]:
# get the proper geography input dimension for model set up 
unique_vals = tf.unique(asfr_training[:, 0]).y
country_geo_dim = np.array(tf.size(unique_vals)).item()
country_geo_dim = country_geo_dim

In [26]:
np.partition(np.unique(asfr_training[:,3]), 3)[:3]

array([0.e+00, 1.e-05, 2.e-05])

In [21]:
# run all country model
for i in range(1,6):
    # Set reproducible seeds per iteration
    np.random.seed(i)
    tf.random.set_seed(i)
    random.seed(i)
    os.environ['PYTHONHASHSEED'] = str(i)

    model_country, loss_info_country = training_functions.run_deep_model(train_prepped, test_prepped, country_geo_dim, 
                                                                         epochs=50, 
                                                                         steps_per_epoch=1405, 
                                                                         lograte=False)

    training_input_features = (tf.convert_to_tensor((asfr_training[:,1] - 1950) / (2015-1950), dtype=tf.float32),  # Normalized year
                            tf.convert_to_tensor(asfr_training[:,2], dtype=tf.float32),  # Age
                            tf.convert_to_tensor(asfr_training[:,0], dtype=tf.float32))  # Geo
    
    test_input_features = (tf.convert_to_tensor((asfr_test[:,1] - 1950) / (2015-1950), dtype=tf.float32),  # Normalized year
                        tf.convert_to_tensor(asfr_test[:,2], dtype=tf.float32),  # Age
                        tf.convert_to_tensor(asfr_test[:,0], dtype=tf.float32))  # Geo

    training_predictions = model_country.predict(training_input_features)

    test_predictions = model_country.predict(test_input_features)

    inputs = np.delete(asfr_training, 3, axis=1)
    training_predictions = np.column_stack((inputs, training_predictions))
    inputs_test = np.delete(asfr_test, 3, axis=1)
    test_predictions = np.column_stack((inputs_test, test_predictions))

    model_country.save(f"../models/dl_model_{i}.keras")

    np.savetxt(f"../data/dl_fitted_{i}.txt", training_predictions)
    np.savetxt(f"../data/dl_forecast_{i}.txt", test_predictions)    

    print(f"Iteration {i} complete")

Epoch 1/50
1405/1405 - 6s - 4ms/step - loss: 0.0024 - val_loss: 7.0921e-04 - learning_rate: 1.0000e-03
Epoch 2/50
1405/1405 - 5s - 3ms/step - loss: 7.4081e-04 - val_loss: 6.3446e-04 - learning_rate: 1.0000e-03
Epoch 3/50
1405/1405 - 4s - 3ms/step - loss: 4.7306e-04 - val_loss: 3.9262e-04 - learning_rate: 1.0000e-03
Epoch 4/50
1405/1405 - 4s - 3ms/step - loss: 2.7625e-04 - val_loss: 2.8674e-04 - learning_rate: 1.0000e-03
Epoch 5/50
1405/1405 - 4s - 3ms/step - loss: 2.0035e-04 - val_loss: 1.9285e-04 - learning_rate: 1.0000e-03
Epoch 6/50
1405/1405 - 5s - 3ms/step - loss: 1.5680e-04 - val_loss: 2.0340e-04 - learning_rate: 1.0000e-03
Epoch 7/50
1405/1405 - 5s - 3ms/step - loss: 1.2976e-04 - val_loss: 1.9185e-04 - learning_rate: 1.0000e-03
Epoch 8/50
1405/1405 - 5s - 3ms/step - loss: 1.1328e-04 - val_loss: 1.4777e-04 - learning_rate: 1.0000e-03
Epoch 9/50
1405/1405 - 5s - 3ms/step - loss: 1.0147e-04 - val_loss: 1.6466e-04 - learning_rate: 1.0000e-03
Epoch 10/50
1405/1405 - 4s - 3ms/step - l